In [2]:
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
import sklearn as sk
import dtale

In [6]:
# df = pd.read_csv("donors.csv", low_memory=False).drop(columns="Unnamed: 0")
# df.to_pickle("donors.pkl")

In [7]:
df = pd.read_pickle("donors.pkl")

In [ ]:
# dtale.show(df)

In [8]:
sample_df = df.iloc[:1000, :].copy()

# EDA

In [9]:
# The following variables indicate the number of
# known times the donor has responded to other
# types of mail order offers.

groupMailOffers = sample_df.loc[:, "MBCRAFT":"PUBOPP"].columns.to_list()

# dtale.show(sample_df[mailOrderOffers])

for col in groupMailOffers:
    sample_df[col] = sample_df[col].fillna(0).astype(np.int16)

In [16]:
groupMDMAUD = [col for col in sample_df.columns.to_list() if "MDMAUD" in col]

In [18]:
sample_df[mdmaudColumns]

,MDMAUD,MDMAUD_R,MDMAUD_F,MDMAUD_A
0,XXXX,X,X,X
1,XXXX,X,X,X
2,XXXX,X,X,X
3,XXXX,X,X,X
4,XXXX,X,X,X
...,...,...,...,...
995,XXXX,X,X,X
996,XXXX,X,X,X
997,XXXX,X,X,X
998,XXXX,X,X,X
